<a href="https://www.kaggle.com/code/singhayush16/horse-health-prediction?scriptVersionId=144277853" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
train=pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')

In [ ]:
train.shape,test.shape

In [ ]:
train.info()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [ ]:
trf1=ColumnTransformer([
    ('impute_temp',SimpleImputer(strategy='most_frequent'),[6]),
    ('impute_pulse',SimpleImputer(strategy='most_frequent'),[7]),
    ('impute_mucous',SimpleImputer(strategy='most_frequent'),[8]),
    ('impute_capillary',SimpleImputer(strategy='most_frequent'),[9]),
    ('impute_pain',SimpleImputer(strategy='most_frequent'),[10]),
    ('impute_peristalsis',SimpleImputer(strategy='most_frequent'),[11]),
    ('impute_abdominal',SimpleImputer(strategy='most_frequent'),[12]),
    ('impute_tube',SimpleImputer(strategy='most_frequent'),[13]),
    ('impute_reflux',SimpleImputer(strategy='most_frequent'),[14]),
    ('impute_rectal',SimpleImputer(strategy='most_frequent'),[16]),
    ('impute_abdomen',SimpleImputer(strategy='most_frequent'),[17]),
    ('impute_abdomo',SimpleImputer(strategy='most_frequent'),[20]),
],remainder='passthrough')

In [ ]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder

In [ ]:
cat_col=train.select_dtypes('object').columns

In [ ]:
for i in cat_col:
    print(i,":",train[i].unique(),":",len(train[i].unique()))
print(len(cat_col))

In [ ]:
trf2=ColumnTransformer([
    ('encode_surgery',OneHotEncoder(drop='first',sparse_output=True,handle_unknown='ignore'),[0]),
    ('encode_mucous',OneHotEncoder(drop='first',sparse_output=True,handle_unknown='ignore'),[8]),
    ('encode_surgical',OneHotEncoder(drop='first',sparse_output=True,handle_unknown='ignore'),[22]),
    ('encode_cp',OneHotEncoder(drop='first',sparse_output=True,handle_unknown='ignore'),[26]),
],remainder='passthrough')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train.sample()

In [ ]:
X=train.drop(['id','outcome'],axis=1)
y=train['outcome']

In [ ]:
le=LabelEncoder()
y_encoded=le.fit_transform(y)

In [ ]:
X_test=test.drop(['id'],axis=1)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y_encoded,random_state=42)

In [ ]:
rf=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('oe',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)),
    ('model',RandomForestClassifier(n_estimators=150,random_state=42))
])

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
rf.score(X_valid,y_valid)

In [ ]:
print(np.mean(cross_val_score(rf,X,y_encoded,cv=5,scoring='accuracy')))

In [ ]:
y_pred=rf.predict(X_test)

In [ ]:
y_pred=le.inverse_transform(y_pred)

In [ ]:
test['outcome']=y_pred
test[['id','outcome']].to_csv('submission_rf.csv',index=False)